# Tokenization

### BertTweet

- fastBPE
- 64K subword

### Twilbert
- SentencePiece (fastBPE)
- 30k subword 

In [22]:
from glob import glob

num_files = 100
tweet_files = glob("../data/filtered_tweets/*.txt")

train_files = tweet_files[:2]

In [23]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False,
    lowercase=False,
)


In [24]:
tokenizer.train(
    train_files,
    vocab_size=40_000,
    min_frequency=2,
    show_progress=True,

    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    limit_alphabet=600,
    wordpieces_prefix="##",
)

In [25]:
from finetune_vs_scratch.preprocessing import special_tokens

tokenizer.add_tokens(special_tokens)

2

In [26]:

tokenizer.save("tokenizador.json")

## Reload

In [27]:
from tokenizers import Tokenizer

tokenizer  = Tokenizer.from_file("tokenizador.json")

In [28]:

tokenizer.encode("@usuario hashtag cagon").tokens

['@usuario', 'hashtag', 'cagon']

In [29]:
import re
len(re.findall(r'[\u4e00-\u9fff]+', " ".join(tokenizer.get_vocab())))

19

In [30]:

tokenizer.encode("@usuario chiques por qué no se hacen ortear").tokens

['@usuario', 'chiques', 'por', 'qué', 'no', 'se', 'hacen', 'or', '##tear']

In [35]:
tokenizer.encode("pibis").tokens

['pib', '##is']

## Loading trained tokenizers

In [37]:
from tokenizers import Tokenizer


tokenizer = Tokenizer.from_file("../models/tokenizers/betito_cased_accents/tokenizer.json")


In [38]:

"@usuario" in tokenizer.get_vocab()

True

In [46]:
tokenizer.encode("@usuario putazo coño troska feminista conurbano kirchnerista").tokens

['@usuario',
 'putazo',
 'coño',
 'tro',
 '##sk',
 '##a',
 'feminista',
 'conurbano',
 'kirchnerista']